## Data Augmentation com Pré-processamento de Imagens

Este notebook abrange o mesmo processo de Data Augmentation explicado anteriormente e detalhado no notebook da Sprint 3 [Notebook Sprint 4](../../SPRINT%203/DATA%20AUGMENTATION/20240524%20-%20Data%20Augmentation.ipynb).

Contudo, alterações foram feitas no pré-processamento. Após a realização de testes com o pré-processamento anterior, observamos que o processo de Data Augmentation resultava em uma redução de cerca de 10% na acurácia do modelo, além de comprometer a qualidade das máscaras preditas.

Além disso, percebemos que o processo de salvamento das imagens foi afetado. A função responsável por esta tarefa estava gerando imagens com um contraste inferior ao necessário, o que prejudicava os processos de convolução.


In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
def load_masks(masks_dir, target_size=(600, 600)):
    masks = []
    ordered_masks = sorted(os.listdir(masks_dir))
    mask_filenames = []
    for mask_name in ordered_masks:
        mask_path = os.path.join(masks_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is not None:
            if mask.shape[:2] != target_size:
                mask = cv2.resize(mask, target_size[::-1], interpolation=cv2.INTER_NEAREST)
            masks.append(mask)
            mask_filenames.append(os.path.splitext(mask_name)[0])
        else:
            print(f"Failed to load mask: {mask_path}")
    return masks, mask_filenames

def load_images(image_dir, target_size=(600, 600)):
    images = []
    ordered_images = sorted(os.listdir(image_dir))
    image_filenames = []
    for image_name in ordered_images:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is not None:
            if image.shape[:2] != target_size:
                image = cv2.resize(image, target_size[::-1])
            images.append(image)
            image_filenames.append(os.path.splitext(image_name)[0])
        else:
            print(f"Failed to load image: {image_path}")
    return images, image_filenames

### Descrição do Processo de Melhoria de Imagem (Enhance Image)

O método *enhance_image* descrito no código realiza uma série de operações para aumentar a qualidade visual das imagens, baseando-se em técnicas bem estabelecidas de processamento de imagem. A principal referência para essas técnicas é o artigo "A Comprehensive Review of Image Enhancement Techniques" de Raman Maini e Himanshu Aggarwal, que oferece uma visão aprofundada sobre as técnicas de melhoria de imagem.

#### Conversão para Escala de Cinza
A conversão de imagens coloridas para escala de cinza simplifica o processamento subsequente ao focar nas intensidades dos pixels, eliminando as informações de cor que podem não ser necessárias para certas análises. Este passo é essencial, pois muitas técnicas de melhoria de imagem funcionam melhor em imagens em escala de cinza.

#### Equalização de Histograma
A equalização de histograma redistribui os valores de intensidade dos pixels para melhorar o contraste global da imagem. Esta técnica é especialmente útil em imagens com iluminação desigual, tornando os detalhes mais visíveis e uniformizando a distribuição de brilho. O artigo de Maini e Aggarwal destaca que a equalização de histograma é eficaz em melhorar a visualização de detalhes em imagens que têm áreas de baixo contraste.

#### Mascaramento de Nitidez
Para melhorar a nitidez da imagem, o método utiliza o mascaramento de nitidez, uma técnica que envolve a suavização da imagem original para reduzir o ruído e, em seguida, a combinação da imagem suavizada com a imagem original para realçar bordas e detalhes finos. Esta técnica melhora a clareza visual da imagem, tornando os detalhes mais proeminentes.

### Justificativas para as Modificações
1. **Conversão para Escala de Cinza**: Simplifica o processamento e foca nas intensidades dos pixels.
2. **Equalização de Histograma**: Melhora o contraste, destacando detalhes em áreas de baixo contraste.
3. **Mascaramento de Nitidez**: Realça bordas e detalhes, aumentando a clareza e definição da imagem.

### Referências

- Maini, R., & Aggarwal, H. (2010). A Comprehensive Review of Image Enhancement Techniques. *Journal of Computing, 2*(3). Disponível em: [arXiv](https://doi.org/10.48550/arXiv.1003.4053)

### Referências adicionais
- OpenCV. (2024). Histogram Equalization. Disponível em: https://docs.opencv.org/3.4/d4/d1b/tutorial_histogram_equalization.html

- Kaggle. (2018). Data Science Bowl 2018 Discussion. Disponível em: https://www.kaggle.com/c/data-science-bowl-2018/discussion/54741#477226

- Neptune. (2023). Image Segmentation Tips and Tricks from Kaggle Competitions. Disponível em: https://neptune.ai/blog/image-segmentation-tips-and-tricks-from-kaggle-competitions

- Dev Genius. (2023). Image Enhancement in Digital Image Processing. Disponível em: https://blog.devgenius.io/image-enhancement-digital-image-processing-21e32f730ced


In [ ]:
class ImageProcessingPipeline:
    def __init__(self, images, masks):
        """
        Inicializa a classe ImageProcessingPipeline com as imagens e máscaras fornecidas.

        Args:
            images (list): Lista de arrays representando imagens.
            masks (list): Lista de arrays representando máscaras correspondentes às imagens.
        """
        self.images = images
        self.masks = masks

    def crop_image(self, image, crop_size=(200, 200)):
        """
        Recorta a imagem em pedaços menores de acordo com o tamanho especificado.

        Args:
            image (array): Array representando a imagem.
            crop_size (tuple): Tamanho do recorte desejado. Padrão é (200, 200).

        Returns:
            list: Lista de recortes da imagem.
        """
        crops = []
        for i in range(0, image.shape[0], crop_size[0]):
            for j in range(0, image.shape[1], crop_size[1]):
                crop = image[i:i+crop_size[0], j:j+crop_size[1]]
                if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
                    crops.append(crop)
        return crops

    def augment_images(self, image):
        """
        Realiza aprimoramentos nas imagens, incluindo rotação e espelhamento.

        Args:
            image (array): Array representando a imagem.

        Returns:
            list: Lista de imagens aprimoradas.
        """
        aug_images = []
        for angle in [0, 90, 180, 270]:
            rotated = self.rotate_image(image, angle)
            aug_images.append(rotated)
            aug_images.append(cv2.flip(rotated, 1))
        return aug_images

    @staticmethod
    def rotate_image(image, angle):
        """
        Rotaciona a imagem pelo ângulo especificado.

        Args:
            image (array): Array representando a imagem.
            angle (int): Ângulo de rotação.

        Returns:
            array: Array representando a imagem rotacionada.
        """
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_NEAREST if len(image.shape) == 2 else cv2.INTER_LINEAR)

    @staticmethod
    def enhance_image(image):
        """
        Aprimora a imagem aplicando equalização de histograma e máscara de nitidez.

        Args:
            image (array): Array representando a imagem.

        Returns:
            array: Array representando a imagem aprimorada.
        """
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image

        equalized = cv2.equalizeHist(gray)
        blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
        sharpened = cv2.addWeighted(equalized, 1.5, blurred, -0.5, 0)

        return sharpened

    def process_and_save_images_and_masks(self, output_dir):
        """
        Processa as imagens e máscaras, realiza aprimoramentos e salva os resultados.

        Args:
            output_dir (str): Diretório de saída para salvar os resultados.
        """
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        count = 0
        for img, mask in zip(self.images, self.masks):
            cropped_images = self.crop_image(img)
            cropped_masks = self.crop_image(mask)
            for crop_img, crop_mask in zip(cropped_images, cropped_masks):
                enhanced_img = self.enhance_image(crop_img)
                augmented_imgs = self.augment_images(enhanced_img)
                augmented_masks = self.augment_images(crop_mask)
                for aug_img, aug_mask in zip(augmented_imgs, augmented_masks):
                    aug_img = Image.fromarray(aug_img)
                    aug_mask = Image.fromarray(aug_mask)
                    output_img_path = os.path.join(output_dir, f'processed_image_{count}.tif')
                    output_mask_path = os.path.join(output_dir, f'processed_mask_{count}.png')
                    print(f"Salvando imagem processada em {output_img_path}")
                    print(f"Salvando máscara processada em {output_mask_path}")
                    aug_img.save(output_img_path)
                    aug_mask.save(output_mask_path)
                    count += 1

    def show_image(self, image):
        """
        Exibe a imagem.

        Args:
            image (array): Array representando a imagem.
        """
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        plt.show()

# Caminhos
masks_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/masks'
image_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/tci_tifs'

# Carregar imagens e máscaras
masks, mask_filenames = load_masks(masks_dir)
images, image_filenames = load_images(image_dir)

# Processar e aumentar imagens e máscaras
output_dir = '/content/drive/Shareddrives/Grupo T de Tech/Data/datasets_sprint4/data_aug'

pipeline = ImageProcessingPipeline(images, masks)
pipeline.process_and_save_images_and_masks(output_dir)

Abaixo estamos aplicando a mesma função que utilzamos no modelo treinado com as imagens geradas no processo acima, para entender e validar que as imagens foram geradas com sucesso

In [ ]:
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size=(256, 256)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

# Listas para armazenar imagens e máscaras pré-processadas
images_processed = []
masks_processed = []

# Obter lista de todos os arquivos de imagens e máscaras
image_files = sorted(glob(os.path.join(output_dir, 'processed_image_*.tif')))
mask_files = sorted(glob(os.path.join(output_dir, 'processed_mask_*.png')))

# Verificar se temos o mesmo número de arquivos de imagem e máscara
assert len(image_files) == len(mask_files), "O número de imagens e máscaras não corresponde!"


# Carregar e pré-processar todas as imagens e máscaras
for img_path, mask_path in zip(image_files, mask_files):
    img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
    images_processed.append(img)
    masks_processed.append(mask)

# Converter para arrays numpy
images_processed = np.array(images_processed)
masks_processed = np.array(masks_processed)

print(f"Shape of images samples: {images_processed.shape}")
print(f"Shape of masks samples: {masks_processed.shape}")


In [ ]:
# Mostrar algumas das imagens processadas
for img in images_processed[:8]:  # Mostra as primeiras 8 imagens processadas
    pipeline.show_image(img)